In [2]:
from astropy.io import fits
from reproject.mosaicking import find_optimal_celestial_wcs
from reproject import reproject_interp

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [12]:
input_file = 'G301_HC3N_Tmb_DR1.fits'

hdu = fits.open(input_file)[0]
data = hdu.data
hdr = hdu.header

hdu_2d = hdu.copy()
hdu_2d.data = hdu_2d.data[0, :, :]
hdu_2d.header['NAXIS'] = 2
del hdu_2d.header['*3*']

wcs_out, shape_out = find_optimal_celestial_wcs(hdu_2d, frame='fk5', auto_rotate=True)
hdr_out_ = wcs_out.to_header()

In [13]:
hdr

SIMPLE  =                    T         /                                        
BITPIX  =                  -32                                                  
NAXIS   =                    3         /                                        
NAXIS1  =                  775         /                                        
NAXIS2  =                  398         /                                        
NAXIS3  =                  801         /                                        
DATAMIN = -0.1285860633850E+02         /                                        
DATAMAX =  0.1389085769653E+02         /                                        
BUNIT   = 'K       '           / Tmb                                            
CTYPE1  = 'GLON-GLS    '               /                                        
CRVAL1  =  0.3010000000000E+03         /                                        
CDELT1  = -0.2638888859183E-02         /                                        
CRPIX1  =  0.3883055184092E+

In [14]:
hdr_out_
hdr_out = hdr.copy()
for key in ['CRPIX1', 'CRPIX2', 'PC1_1', 'PC1_2', 'PC2_1', 'PC2_2', 'CDELT1', 'CDELT2', 'CUNIT1', 'CUNIT2', 'CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'LONPOLE', 'LATPOLE', 'RADESYS', 'EQUINOX']:
    hdr_out[key] = hdr_out_[key]
hdr_out

SIMPLE  =                    T         /                                        
BITPIX  =                  -32                                                  
NAXIS   =                    3         /                                        
NAXIS1  =                  775         /                                        
NAXIS2  =                  398         /                                        
NAXIS3  =                  801         /                                        
DATAMIN = -0.1285860633850E+02         /                                        
DATAMAX =  0.1389085769653E+02         /                                        
BUNIT   = 'K       '           / Tmb                                            
CTYPE1  = 'RA---TAN'                                                            
CRVAL1  =      188.62876662316                                                  
CDELT1  = -0.2638888859183E-02         /                                        
CRPIX1  =        388.3631191

In [ ]:
array = reproject_interp(hdu, hdr_out, return_footprint=False)

In [ ]:
from astropy.io import fits
from reproject.mosaicking import find_optimal_celestial_wcs
from reproject import reproject_interp

input_file = 'G301_HC3N_Tmb_DR1.fits'

hdu = fits.open(input_file)[0]
data = hdu.data
hdr = hdu.header

hdu_2d = hdu.copy()
hdu_2d.data = hdu_2d.data[0, :, :]
hdu_2d.header['NAXIS'] = 2
del hdu_2d.header['*3*']

wcs_out, shape_out = find_optimal_celestial_wcs(hdu_2d, frame='fk5', auto_rotate=True)
hdr_out_ = wcs_out.to_header()

hdr_out = hdr.copy()
for key in ['CRPIX1', 'CRPIX2', 'PC1_1', 'PC1_2', 'PC2_1', 'PC2_2', 'CDELT1', 'CDELT2', 'CUNIT1', 'CUNIT2', 'CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'LONPOLE', 'LATPOLE', 'RADESYS', 'EQUINOX']:
    hdr_out[key] = hdr_out_[key]
hdr_out

array_out = reproject_interp(hdu, hdr_out, return_footprint=False)

hdu_out = fits.PrimaryHDU(array_out, header=hdr_out)
hdu_out.writeto('G301_HC3N_Tmb_DR1_reprojected.fits', overwrite=True)

In [5]:
import dask.array as da
from astropy.io import fits
from reproject.mosaicking import find_optimal_celestial_wcs
from reproject import reproject_interp
import numpy as np

input_file = 'G301_HC3N_Tmb_DR1.fits'

# Open FITS file and retain the HDU
hdul = fits.open(input_file, memmap=True)
hdu = hdul[0]  # Retain reference to the HDU
data = da.from_array(hdu.data, chunks=(1, 100, 100))  # Chunking for Dask
hdr = hdu.header

# Create 2D header for WCS calculation
hdu_2d = hdu.copy()
hdu_2d.data = hdu_2d.data[0, :, :]
hdu_2d.header['NAXIS'] = 2

# Remove 3D-specific header keys
del_keys = [key for key in hdu_2d.header.keys() if '3' in key]
for key in del_keys:
    del hdu_2d.header[key]

# Compute optimal WCS
wcs_out, shape_out = find_optimal_celestial_wcs(hdu_2d, frame='fk5', auto_rotate=True)
hdr_out_ = wcs_out.to_header()

# Copy original header and update with new projection
hdr_out = hdr.copy()
for key in ['CRPIX1', 'CRPIX2', 'PC1_1', 'PC1_2', 'PC2_1', 'PC2_2', 'CDELT1', 'CDELT2',
            'CUNIT1', 'CUNIT2', 'CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'LONPOLE', 'LATPOLE',
            'RADESYS', 'EQUINOX']:
    hdr_out[key] = hdr_out_[key]

# Ensure all header values are properly formatted
for key, value in hdr_out.items():
    if isinstance(value, bool):  # FITS headers do not support bools directly
        hdr_out[key] = 'T' if value else 'F'
    elif isinstance(value, bytes):  # Ensure string values are in proper format
        hdr_out[key] = value.decode('utf-8')

# Create an empty output FITS file
output_file = 'G301_HC3N_Tmb_DR1_reprojected.fits'
hdu_out = fits.PrimaryHDU(header=hdr_out)
hdu_out.writeto(output_file, overwrite=True)

# Open for updating with memory mapping
hdu_out = fits.open(output_file, mode='update', memmap=True)
output_array = hdu_out[0].data

# Reproject using Dask
rslt = reproject_interp(hdu, hdr_out, output_array=output_array, return_footprint=False)

hdu_out.flush()
hdul.close()  # Close the original FITS file
hdu_out.close()  # Close the output FITS file

KeyboardInterrupt: 